In [1]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cluster import KMeans, k_means_, DBSCAN, AgglomerativeClustering
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_distances

import hdbscan
from gensim import models
from gensim.corpora import mmcorpus, Dictionary
from gensim.models import lsimodel, ldamodel, tfidfmodel, rpmodel, logentropy_model, TfidfModel, LsiModel, LdaModel
from gensim import matutils, corpora
from gensim.models.doc2vec import TaggedDocument
from gensim.models import doc2vec
from gensim.models.word2vec import Word2Vec
from sklearn.svm import SVC

import sys
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from konlpy.utils import pprint
mecab = Mecab()
%matplotlib inline

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
dataDict = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
print (len(dataDict))

4722


In [3]:
keywordsDict = pickle.load(open('./data/pre_data/keywords/keywords_daum.pickled','rb'))

In [4]:
for idx in dataDict:
    dataDict[idx]['extracted_keywords'] = keywordsDict[idx]

In [5]:
df_daum = pd.DataFrame.from_dict(dataDict, orient='index')
df_daum['date'] = pd.to_datetime(df_daum['date']).dt.date
df_daum.reset_index(inplace = True)
df_daum.rename(columns={'index':'id'}, inplace=True)
print (df_daum.shape)
df_daum.head()

(4722, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a2a61bf588c13481c229d1e,뉴스,2017-12-07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{건강, 연기, 세보 효과, 공장, 남성들, 부작용, 지역}"
1,5a2a61bf588c13481c229d1f,뉴스,2017-12-07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{건강, 자세, 식후, 디스크, 낮잠, 점심 식사, 커피, 치아, 입냄새, 철분}"
2,5a2a61bf588c13481c229d20,뉴스,2017-12-07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{주변, 앙심, 성관계, 지인, 철원, 범행, 진술, 아들, 경찰, 남편}"
3,5a2a61bf588c13481c229d21,뉴스,2017-12-07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{유통업체, 매장, 네이처리퍼블릭, 신영자 이사장, 검찰, 혐의, 징역, 롯데}"
4,5a2a61bf588c13481c229d22,뉴스,2017-12-07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{남자친구, 과자, 범행, 쪽지, 유치장, 폭행, 혐의, 경찰}"


In [7]:
def nav_tokenizer(corpus):
    pos = mecab.pos(corpus)
    res = [x[0] for x in pos if (x[1] == u'VV' or x[1] == u'VA' or x[1] == u'NNB' or x[1] == u'NNP' or x[1] == u'NNG')]
    return res

In [8]:
def remove_stopwords(words, stopwords):
    res = [x for x in words if x not in stopwords]
    res = [x for x in words if len(x) != 1]
    return res

In [9]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

In [11]:
d2v_docs = [nav_tokenizer(x) for x in textInfo]
d2v_docs = [remove_stopwords(x, (stopwords)) for x in d2v_docs]

### word2vec

In [12]:
w2v_model = Word2Vec(d2v_docs, sg=1, size=3000,window=8, workers = 2, iter =10)
w2v_model.init_sims(replace = True)

In [13]:
print(w2v_model.most_similar('신혜선'))

[('박시후', 0.8729689121246338), ('서은수', 0.8265682458877563), ('황금빛', 0.8153809309005737), ('내인생', 0.7813573479652405), ('소현경', 0.7713840007781982), ('천호진', 0.760439932346344), ('셰어하우스', 0.7490668296813965), ('나영희', 0.7439956068992615), ('유인영', 0.7405895590782166), ('김형석', 0.7404094934463501)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [14]:
print(w2v_model.most_similar('손흥민'))

[('토트넘', 0.7790887355804443), ('에릭센', 0.7547650933265686), ('스토크시티', 0.7305456399917603), ('델레', 0.7259833812713623), ('스토크', 0.7120370864868164), ('해트트릭', 0.6984328627586365), ('득점포', 0.6951755285263062), ('아포엘', 0.685232400894165), ('케인', 0.6844925880432129), ('라멜라', 0.6839351654052734)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [15]:
print(w2v_model.most_similar('황금빛내인생'))

[('대역', 0.8592950105667114), ('야화', 0.8572156429290771), ('이진', 0.8570830225944519), ('이다인', 0.8525866270065308), ('재벌남', 0.8427147269248962), ('빅픽처', 0.837897002696991), ('이태성', 0.8346164226531982), ('정소영', 0.8324530124664307), ('짝사랑', 0.8255244493484497), ('원민', 0.8254272937774658)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [16]:
print(w2v_model.most_similar('홍준표'))

[('자유한국당', 0.7770987153053284), ('김무성', 0.7440187931060791), ('비박', 0.7386003732681274), ('친박', 0.7275629639625549), ('대한애국당', 0.7220845222473145), ('조원진', 0.7202844619750977), ('천정배', 0.7076305747032166), ('비홍', 0.7073788046836853), ('이완구', 0.7055678367614746), ('출마설', 0.6997513771057129)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [17]:
print(w2v_model.most_similar('문재인'))

[('문재', 0.6807799935340881), ('방중', 0.6437293291091919), ('국빈방문', 0.6429750919342041), ('대통령', 0.6293697357177734), ('노무현', 0.6202112436294556), ('조원진', 0.6158163547515869), ('조찬', 0.6130086779594421), ('지지자', 0.6049723029136658), ('헌재소장', 0.6043819189071655), ('국빈', 0.6029701232910156)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [18]:
print(w2v_model.most_similar('노무현'))

[('김영삼', 0.8029703497886658), ('서거', 0.785525918006897), ('이회창', 0.7608038187026978), ('김대중', 0.752109169960022), ('변양균', 0.7410458326339722), ('정치권력', 0.7370416522026062), ('장준하', 0.7336248755455017), ('국정운영', 0.7330576181411743), ('연설문', 0.7314356565475464), ('현철', 0.7287083864212036)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [19]:
print(w2v_model.most_similar('박근혜'))

[('탄핵', 0.742803692817688), ('파면', 0.7055424451828003), ('정권', 0.6797739267349243), ('이명박', 0.6779638528823853), ('법치', 0.6717170476913452), ('탄핵안', 0.670433521270752), ('박근혜정부', 0.6675702929496765), ('헌재소장', 0.6638773679733276), ('국정조사', 0.661862850189209), ('비선실세', 0.6591272354125977)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [20]:
print(w2v_model.most_similar('국정원'))

[('원세훈', 0.7966267466545105), ('상납', 0.7811402678489685), ('특활비', 0.7810839414596558), ('추명호', 0.7770677208900452), ('국정원장', 0.773865818977356), ('대선개입', 0.7690730094909668), ('민정수석', 0.7547343373298645), ('이석수', 0.7522993683815002), ('문건', 0.7487953901290894), ('화이트리스트', 0.7438424229621887)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [21]:
print(w2v_model.most_similar('이낙연'))

[('소방점검', 0.8638492822647095), ('정부서울청사', 0.8207470178604126), ('운영위', 0.8128197193145752), ('행안부', 0.8116700649261475), ('우원식', 0.8074386119842529), ('속도전', 0.8000713586807251), ('김영주', 0.7998073101043701), ('김부겸', 0.7988554239273071), ('성토', 0.7974178791046143), ('단양', 0.7927007675170898)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [22]:
print(w2v_model.most_similar('이국종'))

[('아주대병원', 0.8410766124725342), ('중증외상센터', 0.8350719213485718), ('귀순병', 0.8338961005210876), ('문종환', 0.8278180360794067), ('외상센터', 0.8213338851928711), ('외상외과', 0.7983059883117676), ('권준식', 0.7972826361656189), ('외상', 0.7809426188468933), ('피눈물', 0.7763470411300659), ('오청', 0.760673999786377)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [23]:
print(w2v_model.most_similar('김성태'))

[('원내대표', 0.7721876502037048), ('예결위', 0.7018588781356812), ('김용태', 0.7005733251571655), ('정우택', 0.6919291019439697), ('우원식', 0.6918641924858093), ('하태경', 0.691216230392456), ('결속', 0.6873055696487427), ('집권당', 0.683763861656189), ('김무성', 0.6761471629142761), ('야합', 0.6724929809570312)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [24]:
print(w2v_model.most_similar('더불어민주당'))

[('우원식', 0.866391658782959), ('정우택', 0.8571557998657227), ('간사인', 0.8404224514961243), ('안민석', 0.8352137804031372), ('김용태', 0.830752968788147), ('추미애', 0.830359935760498), ('안산시', 0.8234376311302185), ('반대표', 0.8225874900817871), ('오신환', 0.8210635781288147), ('예결위', 0.8165100812911987)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [25]:
print(w2v_model.most_similar('새누리당'))

[('한나라당', 0.8061717748641968), ('서청원', 0.7995332479476929), ('김무성', 0.7837322950363159), ('시사저널', 0.7817878723144531), ('탈당', 0.7797075510025024), ('홍문종', 0.7752557992935181), ('이혜훈', 0.7721641063690186), ('김세연', 0.7657749056816101), ('이수담', 0.76246178150177), ('샛문', 0.7617939710617065)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [26]:
print(w2v_model.most_similar('자유한국당'))

[('홍준표', 0.7770986557006836), ('한국당', 0.7758086323738098), ('김무성', 0.7414132356643677), ('추미애', 0.7401326298713684), ('원내대표', 0.7346935272216797), ('오신환', 0.7272235155105591), ('홍문종', 0.7223349213600159), ('정우택', 0.7206293940544128), ('조원진', 0.718337893486023), ('한선교', 0.7162386775016785)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [27]:
print(w2v_model.most_similar('국민의당'))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('안철수', 0.8443269729614258), ('정당', 0.8043214082717896), ('천정배', 0.7852683663368225), ('반대파', 0.7852445244789124), ('당원', 0.7763022780418396), ('당내', 0.7757860422134399), ('중진', 0.7707997560501099), ('바른정당', 0.7703557014465332), ('양당', 0.765972375869751), ('유성엽', 0.7627339363098145)]


In [28]:
print(w2v_model.most_similar('안철수'))

[('국민의당', 0.844326913356781), ('바른정당', 0.8061538934707642), ('유승민', 0.7883384227752686), ('반대파', 0.7801316976547241), ('중진', 0.7776541113853455), ('손학규', 0.775761604309082), ('대표직', 0.7646279335021973), ('간신배', 0.7638431191444397), ('당내', 0.7629367709159851), ('창당', 0.7625021934509277)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [29]:
print(w2v_model.most_similar('유승민'))

[('바른정당', 0.8750512599945068), ('김무성', 0.8023240566253662), ('안철수', 0.7883385419845581), ('박영선', 0.7879614233970642), ('컨설턴트', 0.7811828255653381), ('김세연', 0.7801530957221985), ('적합도', 0.7795282602310181), ('대표직', 0.772723913192749), ('이학재', 0.7711766958236694), ('샛문', 0.7707030773162842)]


C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [30]:
print(w2v_model.most_similar('박지성'))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('차범근', 0.7631536722183228), ('긱스', 0.7165627479553223), ('트라', 0.7106199264526367), ('프랑크푸르트', 0.6996489763259888), ('서포터즈', 0.6951666474342346), ('오노', 0.6782155632972717), ('레드냅', 0.6741582155227661), ('유스', 0.6733450889587402), ('명장', 0.6684054136276245), ('설가', 0.6652443408966064)]


In [31]:
w2v_model.save('./model/word2vec_category_size3000_window8_iter10_by_mecab.w2v')

In [73]:
embedding_matrix = np.zeros((len(w2v_model.wv.vocab), 3000))
for i in range(len(w2v_model.wv.vocab)):
    embedding_vector = w2v_model.wv[w2v_model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [74]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, embeddings, merge

In [75]:
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
# input words - in this case we do sample by sample evaluations of the similarity
valid_word = Input((1,), dtype='int32')
other_word = Input((1,), dtype='int32')
# setup the embedding layer
embeddings = Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1],
                      weights=[embedding_matrix])
embedded_a = embeddings(valid_word)
embedded_b = embeddings(other_word)
similarity = merge([embedded_a, embedded_b], mode='cos', dot_axes=2)
# create the Keras model
k_model = Model(input=[valid_word, other_word], output=similarity)

def get_sim(valid_word_idx, vocab_size):
    sim = np.zeros((vocab_size,))
    in_arr1 = np.zeros((1,))
    in_arr2 = np.zeros((1,))
    in_arr1[0,] = valid_word_idx
    for i in range(vocab_size):
        in_arr2[0,] = i
        out = k_model.predict_on_batch([in_arr1, in_arr2])
        sim[i] = out
    return sim

# now run the model and get the closest words to the valid examples
for i in range(valid_size):
    valid_word = w2v_model.wv.index2word[valid_examples[i]]
    top_k = 8  # number of nearest neighbors
    sim = get_sim(valid_examples[i], len(w2v_model.wv.vocab))
    nearest = (-sim).argsort()[1:top_k + 1]
    log_str = 'Nearest to %s:' % valid_word
    for k in range(top_k):
        close_word = w2v_model.wv.index2word[nearest[k]]
        log_str = '%s %s,' % (log_str, close_word)
    print(log_str)

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':
/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("me...)`
  


Nearest to 선수: 주전급, 외국인선수, 호령, 김경언, 김정혁, 정재훈, 휘문고, 강영식,
Nearest to 올해: 선수상, 조정훈, 신재영, 3관왕, 퓨쳐스리그, 전관왕, 임정우, 손승락,
Nearest to 이야기: 들려주, 박문수, 진시황, 사석, 쑥스럽, 스릴러, 어사, 이승,
Nearest to 일본: 도쿄, 스포츠호치, 지바, 바히드, 니치, 스포츠파크, 윤덕여, 할릴호지치,
Nearest to 사이: 월화드라마, 김진원, 버사, 원진아, 쇼핑몰, 강두, 박시은, 스며들,
Nearest to 사실: 진술서, 증거자료, 실토, 친자, 차명, 혐의점, 강순덕, 명예훼손죄,
Nearest to 대표: 홍준표, 대표직, 안철수, 창당, 하태경, 송아, 친안, 인재,
Nearest to 투수: 필승조, 좌완, 불펜, 임찬규, 배영수, 좌완투수, 함덕주, 피어밴드,
Nearest to 오후: 인터콘티넨탈호텔, 메이플홀, 여의도동, 4시, 하동, 입관식, 고양시, 임헌정,
Nearest to 기자: 조현, 아나, 엄기찬, 임창, 다희, 정형근, 설희, 최은지,
Nearest to 롯데: 자이언츠, 린드블럼, 민병헌, 강민호, 사도스키, 윤길현, 브룩스, 레일리,
Nearest to 친구: 치맥, 페트리, 로빈, 놀이동산, 문상돈, 창덕궁, 코펜하겐, 유람선,
Nearest to 관계자: 내막, 다각도, 성산읍, 플럼, 확답, 서울시교육청, 시공업체, 광역수사대,
Nearest to 공개: 허민, 축가, 홈즈, 절친, 빽가, 윌엔터테인먼트, 강연자, 포로수용소,
Nearest to 때문: 싫어하, 카테터, 덜하, 두뇌, 졸음, 합병증, 내키, 제트기류,
Nearest to 지난해: 영업이익, 뒷걸음질, 실질소득, 출고량, 선두주자, 국세, 수출액, 임정우,


In [ ]:
w2v_model.wv.